In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [32]:
url_inicial='http://tzintzun.umich.mx/index.php/TZN/issue/archive/2'
url_root= 'http://tzintzun.umich.mx/index.php/TZN/issue/archive/2'
r = requests.get(url_inicial)

In [33]:
soup = BeautifulSoup(r.text, 'html.parser')

In [34]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['http://tzintzun.umich.mx/index.php/TZN/issue/view/26',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/27',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/28',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/29',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/30',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/31',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/32',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/33',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/34',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/35',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/36',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/37',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/38',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/39',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/40',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/41',
 'http://tzintzun.umich.mx/index.php/TZN/issue/view/42',
 'http://tzintzun.umich.mx/inde

In [35]:
    vol2 =[]
    for i in vol: 
        url_inicial1=i 
        r1 = requests.get(url_inicial1)
        soup1 = BeautifulSoup(r1.text, 'html.parser')
        box1 = soup1.find('ul', class_='cmp_article_list articles') 
        volumen1=box1.findAll('div', class_='obj_article_summary')
        vol1 = [x.find('a').get('href')for x in volumen1]
        vol2+=vol1

In [36]:
vol2

['http://tzintzun.umich.mx/index.php/TZN/article/view/204',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/205',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/206',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/207',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/208',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/213',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/214',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/215',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/216',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/221',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/222',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/223',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/224',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/225',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/226',
 'http://tzintzun.umich.mx/index.php/TZN/article/view/232',
 'http://tzintzun.umich.mx/index.php/TZN

In [20]:
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [21]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://tzintzun.umich.mx/index.php/TZN/issue/archive/2


In [22]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

138

In [23]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [24]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Tzintzun. Revista de Estudios Históricos'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Historia'
    #url='http://www.rmiq.org/ojs311/index.php/rmiq/issue/view/65'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('section', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [25]:
list_scraper=list_scraper[0:157]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [26]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Los campesinos mexicanos y la guerra de Indepe...,ResumenEste artículo revisa la historiografía ...,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
1,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Revisión historiográfica de los insurgentes-re...,ResumenEn este trabajo se hace un exámen a la ...,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
2,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Uso y abuso de los indígenas atalayeros en el ...,ResumenEl trabajo de los â€œindígenas atalayer...,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
3,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Continuidades en el discurso intelectual y pol...,ResumenEste artículo analiza el contenido del ...,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
4,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Mirada crítica a la historiografía cubana en t...,ResumenEn esta mirada a la historiografía cuba...,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
...,...,...,...,...,...,...,...,...
133,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Las relaciones fronterizas entre México y Esta...,None,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
134,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Pablo Beaumont y el Códice de Tzintzuntzan: Do...,None,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
135,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,Legislar y comunicar. Los inicios de la telefo...,None,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...
136,Tzintzun. Revista de Estudios Históricos,IV. Humanidades y Ciencias de la Conducta,Historia,El posmodernismo: La historia subversivaÂ,None,,http://tzintzun.umich.mx/index.php/TZN/article...,http://tzintzun.umich.mx/index.php/TZN/article...


In [27]:
df_catalogo.to_csv('Revista09.03.csv', index=False)